In [ ]:
#Opitimized for display in Voila - Chrome, Editing in JupyterLab

In [1]:
#Magic for the temperature plot, calling it twice makes the plot more stable
%matplotlib widget
%matplotlib widget

In [2]:
#Packages and corresponding versions:
#miniconda
#pandas: 1.5.3
#ika: latest
#matplotlib
#jupyter latest
#numpy 1.26.4
#python 3.10
#tkinter latest
#TODO: move generated data from autsave to state file ---> smaller files
#TODO limit log file size
#remove ika dependecny
#look into d2yy ftcycle to reset usb port in case of critical failure
#TODO: remove pandas dependency (save files -->csv ( chatgpt can do it easyily) + ika import) ---> hugely reduced dependecy list

In [3]:
#import packages
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import numpy as np 
import threading
import time
from datetime import datetime
import tkinter as tk
from tkinter import ttk
import datetime
import logging
import configparser
from tkinter import filedialog
import math
import os,shutil
import pandas as pd
import logging.handlers
import serial.tools.list_ports
from ika.magnetic_stirrer import MockMagneticStirrer,MagneticStirrer
from collections.abc import Callable
from typing import Any

In [4]:
#logging decorator, for simple functions
def log(success_message: str, error_message: str) -> Callable[[Callable[..., Any]], Callable[..., Any]]:
    def decorator(func: Callable[..., Any]) -> Callable[..., Any]:
        def wrapper(*args: Any, **kwargs: Any) -> Any:
            try:
                result = func(*args, **kwargs)
                logging.info(f"{datetime.datetime.now()}   {success_message}")
                return result
            except Exception as e:
                logging.error(f"{datetime.datetime.now()}   {error_message}: {e}")
        return wrapper
    return decorator

In [5]:
#check if autosaves and logs folders exist, create them if needed
def create_dirs() -> None:
    try:
        if not os.path.isdir('autosaves'):
            os.makedirs('autosaves')
        if not os.path.isdir('logs'):
            os.makedirs('logs')
    except:
        pass
create_dirs()

In [6]:
#read data from client implementation for serverside UNUSED:
def read_data_from_client():
    initate_control = False
    read_data = pd.read_csv('sent_data.csv')
    if read_data["Manual stirring"][0] != -1:
        if str(read_data["Manual stirring"][0]) == "True":
            flip_stirring_on()
        else:
            flip_stirring_off()
    if read_data["Manual heating"][0] != -1:
        if str(read_data["Manual heating"][0]) == "True":
            flip_heating_on()
        else:
            flip_heating_off()
    if int(read_data["Manual heating"][1]) != -1:
        heater_temp_slider.value = int(read_data["Manual heating"][1])
    if int(read_data["Manual stirring"][1]) != -1:
        stirrer_speed_slider.value = int(read_data["Manual stirring"][1])
    if read_data["Manual control"][0] != -1:
        if str(read_data["Manual control"][0]) == "Start Control": 
            if Start_control_button.description == "Start Control":
                pass
                #no change
            else:
                #stopping control
                Start_control_button_clicked('b')
                logging.info(str(datetime.datetime.now()) + '   ' + 'Control stopped by outside user')
        else:
            initate_control = True
    if int(read_data["Manual set time"][0]) != -1:
        if initate_control:
            if Start_control_button.description == "Start Control":
                for i in range(int(len(read_data['Manual set temp'])/2)):
                    Temp_box[i].value = read_data['Manual set temp'][(i*2)+1]
                    if i > 0:
                        Time_to_complete_box[i].value= read_data['Manual set time'] [(i*2)-1]/60
                        Time_to_hold_box[i].value= read_data['Manual set time'] [(i*2)]/60
                time.sleep(1)
                on_generate_profile_button_clicked("b")
                time.sleep(1)
                Start_control_button_clicked('b')
            else:
                Start_control_button_clicked('b')
                time.sleep(1)
                for i in range(int(len(read_data['Manual set temp'])/2)):
                    Temp_box[i].value = read_data['Manual set temp'][(i*2)+1]
                    if i > 0:
                        Time_to_complete_box[i].value= read_data['Manual set time'] [(i*2)-1]/60
                        Time_to_hold_box[i].value= read_data['Manual set time'] [(i*2)]/60
                time.sleep(1)
                on_generate_profile_button_clicked("b")
                time.sleep(1)
                Start_control_button_clicked('b')
        else:
            for i in range(int(len(read_data['Manual set temp'])/2)):
                Temp_box[i].value = read_data['Manual set temp'][(i*2)+1]
                if i > 0:
                    Time_to_complete_box[i].value= read_data['Manual set time'] [(i*2)-1]/60
                    Time_to_hold_box[i].value= read_data['Manual set time'] [(i*2)]/60
            time.sleep(1)
            on_generate_profile_button_clicked("b")
    os.remove('sent_data.csv')

In [7]:
#reload data from previous measurment
@log(success_message='Succesfully reloaded previous save', error_message='Error while reloading data from save file')
def reload_data_from_previous_measurment(location_prev_state='autosaves/previous_state.csv'):
    global Logged_time_points, Logged_temperature_points,measurement_date, Start_point
    global Generated_time_points,Generated_temperature_points, Set_time_points, Set_temperature_points
    global Perfect_time_points,Perfect_temperature_points, number_of_inputs
    temperatures = []
    times = []
    def pad_list(lst, pad_value, max_length):
        return lst + [pad_value] * (max_length - len(lst))

    read_prev_state = pd.read_csv(str(location_prev_state))
    read_data = pd.read_csv("autosaves/autosave"+str(read_prev_state['date_of_measurment'][0])[:12] +".csv")
    if str(read_prev_state["Heatin_stirring_states"][0]) == "True":
        flip_stirring_on()
    else:
        flip_stirring_off()
    if str(read_prev_state["Heatin_stirring_states"][2]) == "True":
        flip_heating_on()
    else:
        flip_heating_off()
    heater_temp_slider.value = int(read_prev_state["Heatin_stirring_states"][3])-1
    stirrer_speed_slider.value = int(read_prev_state["Heatin_stirring_states"][1])-1
    heater_temp_slider.value = int(read_prev_state["Heatin_stirring_states"][3])
    stirrer_speed_slider.value = int(read_prev_state["Heatin_stirring_states"][1])
    try:
        for i in range(int(len(read_prev_state["Temp_box_settings"])/2)):
            Temp_box[i].value = read_prev_state["Temp_box_settings"][(i*2)+1]
            if i > 0:
                Time_to_complete_box[i].value= read_prev_state['Time_box_settings'] [(i*2)-1]/60
                Time_to_hold_box[i].value=read_prev_state['Time_box_settings'] [(i*2)]/60
        #time.sleep(1)
        #on_generate_profile_button_clicked("b")
    except:
        logging.error(str(datetime.datetime.now())+ '     ' +   'error while reloading input box data, maybe number of inputs changed compared to save?')
    Logged_time_points =  [float(x*60) for x in read_data["Measured time (minutes)"] if x is not None and not math.isnan(x)]
    Logged_temperature_points = [float(x) for x in read_data["Measured temperature (C)"] if x is not None and not math.isnan(x)]
    measurement_date = [x for x in read_data["Measurement Date"] if x is not None and (not isinstance(x, str) or x.strip() != "")]
    Generated_time_points = [float(x)*60 for x in read_data["Generated heating points time (minutes)"] if x is not None and not math.isnan(x)]
    Generated_temperature_points = [float(x) for x in read_data["Generated heating points temperature (C)"] if x is not None and not math.isnan(x)]
    Perfect_time_points = [float(x)*60 for x in read_data["Temperature profile time (minutes)"] if x is not None and not math.isnan(x)]
    Perfect_temperature_points = [float(x) for x in read_data["Temperature profile temperature (C)"] if x is not None and not math.isnan(x)]
    logging.info(str(datetime.datetime.now()) + '   ' + 'Succesfully reloaded previous state')
    Start_point = read_prev_state["Current point"][0]
    max_length = max(
    len(Logged_time_points),
    len(Logged_temperature_points),
    )
    if len(Logged_time_points) < max_length:
        Logged_time_points = pad_list(Logged_time_points, Logged_time_points[-1], max_length)
    if len(Logged_temperature_points) < max_length:
        Logged_temperature_points = pad_list(Logged_temperature_points, Logged_temperature_points[-1], max_length)
    if len(measurement_date) < max_length:
        measurement_date = pad_list(measurement_date, measurement_date[-1], max_length)
    while len(measurement_date) > max_length:
        measurement_date.pop()
    Logged_temperature_points.pop()
    Set_time_points = []
    Set_time_points.append(0)
    Set_temperature_points = []
    for i in np.arange(0,number_of_inputs):       
        temperatures.append(Temp_box[i].value)
        temperatures.append(Temp_box[i].value)
        if i != 0:
            times.append(Time_to_complete_box[i].value*60)
        times.append(Time_to_hold_box[i].value*60)
    Set_temperature_points = temperatures 
    Set_time_points = times


In [8]:
#create config file
def create_config(T_No=6, Load_prev=True, interval=1,save_interval=30,selected_save ='Last save', def_port = 'COM3', try_all = True,debug_mode=False):
    config = configparser.ConfigParser()
 
    # Add sections and key-value pairs
    config['General'] = {'number_of_temperature_points': T_No, 'keep_data_from_previous_setup': Load_prev, 'interval' : interval,'save_interval' :save_interval, 'selected_save':selected_save }
    config['Port_setup'] = {'default_port': def_port, 'try_all_ports': try_all, 'debug_mode':debug_mode} 
    # Write the configuration to a file
    with open('config.ini', 'w') as configfile:
        config.write(configfile)

In [9]:
#read config file
def read_config() -> dict[str, str]:
    # Create a ConfigParser object
    config = configparser.ConfigParser()
 
    # Read the configuration file
    config.read('config.ini')
 
    # Access values from the configuration file
    number_of_temperature_poin = config.get('General', 'number_of_temperature_points')
    keep_data_from_previous_setu = config.getboolean('General', 'keep_data_from_previous_setup')
    interva = config.get('General', 'interval')
    save_interva = config.get('General', 'save_interval')
    selected_sav = config.get('General', 'selected_save')
    default_por = config.get('Port_setup', 'default_port')
    try_al = config.getboolean('Port_setup', 'try_all_ports')
    debug_mod =  config.getboolean('Port_setup', 'debug_mode')
    # Return a dictionary with the retrieved values
    config_values = {
        'No_of_boxes': number_of_temperature_poin,
        'load_previous': keep_data_from_previous_setu,
        'logging_interval' : interva,
        'save_interval': save_interva,
        'selected_save': selected_sav,
        'default_port' : default_por,
        'try_all_ports': try_al,
        'debug_mode': debug_mod
    }
 
    return config_values

In [10]:
#Save time of measuremnt
date_of_measurment = (str(datetime.datetime.now())[:16])
date_of_measurment=date_of_measurment.replace('-','')
date_of_measurment=date_of_measurment.replace(':','')
date_of_measurment=date_of_measurment.replace(' ','')

In [11]:
#create logfile, rename previous state
logfile_name = "logs/logfile" + date_of_measurment + ".log"
logging.basicConfig(filename=logfile_name, level=logging.INFO)
logging.info(str(datetime.datetime.now()) + '   ' + 'Logging started')
logging.error(str(datetime.datetime.now()) + '   ' + 'Error test')
if os.path.exists('autosaves/Current_state.csv'):
    try:
        read_prev_state = pd.read_csv("autosaves/previous_state.csv")
        try:
            os.remove("autosaves/previous_state"+str(read_prev_state['date_of_measurment'][0])[:12] +".csv")
            logging.error(str(datetime.datetime.now()) + '   ' + 'Overwriting prev state save file')
        except:
            pass
        os.rename('autosaves/previous_state.csv',"autosaves/previous_state"+str(read_prev_state['date_of_measurment'][0])[:12] +".csv")
    except Exception as e:
        logging.error(str(datetime.datetime.now()) + '   ' + 'Error while reading prev state ' + str(e))

    try:    
        os.rename('autosaves/Current_state.csv','autosaves/previous_state.csv')
    except Exception as e:
        logging.error(str(datetime.datetime.now()) + '   ' + 'Error while renaming prev state ' + str(e))
else:
    try:
        #shutil.copy('autosaves/previous_state.csv','autosaves/Current_state.csv')
        pass
    except Exception as e:
        logging.error(str(datetime.datetime.now()) + '   ' + 'Error while renaming current state ' + str(e))   

In [12]:
#Autosave file to autosaves folder with name: autsaveDATE.csv
def autosave(file_path=str("autosaves/autosave" + date_of_measurment + ".csv")) -> None:
    global Logged_time_points, interval, Logged_temperature_points,Perfect_time_points,Perfect_temperature_points, measurement_date
    global Generated_time_points,Generated_temperature_points
    #make sure lists are the same length
    max_length = max(
        len(Generated_time_points),
        len(Generated_temperature_points),
        len(Perfect_time_points),
        len(Perfect_temperature_points),
        len(Logged_time_points),
        len(Logged_temperature_points),
        len(measurement_date)
    )
    def pad_list(lst, pad_value, max_length):
        return lst + [pad_value] * (max_length - len(lst))
    Generated_time_points_padded = pad_list(Generated_time_points, None, max_length)
    Generated_temperature_points_padded = pad_list(Generated_temperature_points, None, max_length)
    Perfect_time_points_padded = pad_list(Perfect_time_points, None, max_length)
    Perfect_temperature_points_padded = pad_list(Perfect_temperature_points, None, max_length)
    Logged_time_points_padded = pad_list(Logged_time_points, None, max_length)
    Logged_temperature_points_padded = pad_list(Logged_temperature_points, None, max_length)
    measurement_date_padded = pad_list(measurement_date, None, max_length)
    #generate csv
    df =pd.DataFrame({
        "Generated heating points time (minutes)": [x / 60 if x is not None else None for x in Generated_time_points_padded],
        "Generated heating points temperature (C)": Generated_temperature_points_padded,
        "Temperature profile time (minutes)": [x / 60 if x is not None else None for x in Perfect_time_points_padded],
        "Temperature profile temperature (C)": Perfect_temperature_points_padded,
        "Measured time (minutes)": [x / 60 if x is not None else None for x in Logged_time_points_padded],
        "Measured temperature (C)": Logged_temperature_points_padded,
        "Measurement Date": measurement_date_padded
    })
    #save csv
    df.to_csv(file_path, index=False)
    df = [] 
    #save state (separate to keep file sizes smaller)
    autosave_state()

In [13]:
#autosave current setup
def autosave_state():
    global Set_temperature_points,Set_time_points,date_of_measurment, current_point_data
    Heatin_stirring_states = []
    date_of_measurment_list=["a"]
    date_of_measurment_list[0] = date_of_measurment
    Heatin_stirring_states.append(stirrer_button.value)
    Heatin_stirring_states.append(stirrer_speed_slider.value)
    Heatin_stirring_states.append(heater_button.value)
    Heatin_stirring_states.append(heater_temp_slider.value)
    Heatin_stirring_states.append(info_widget.value)
    Heatin_stirring_states.append(Start_control_button.description)
    max_length = max(
        len(Heatin_stirring_states),
        len(Set_temperature_points),
        len(Set_time_points),
        len(current_point_data)
    )
    def pad_list(lst, pad_value, max_length):
        return lst + [pad_value] * (max_length - len(lst))
    Heatin_stirring_states = pad_list(Heatin_stirring_states, None, max_length)
    Set_time_points_padded = pad_list(Set_time_points, None, max_length)
    Set_temperature_points_padded = pad_list(Set_temperature_points, None, max_length)
    date_of_measurment_padded =pad_list(date_of_measurment_list  , None, max_length)
    current_point_data_padded = pad_list(current_point_data  , None, max_length)
    df =pd.DataFrame({
        "Temp_box_settings": Set_temperature_points_padded,
        "Time_box_settings": Set_time_points_padded,
        "Heatin_stirring_states": Heatin_stirring_states,
        "date_of_measurment": date_of_measurment_padded ,
        "Current point": current_point_data_padded
    })
    df.to_csv('autosaves/Current_state.csv', index=False)
    df = []
    logging.info(str(datetime.datetime.now()) + '   ' + "State saved succesfully")

In [14]:
#Functions of heating,stirring, reloading previous state buttons
@log(success_message='Heating Turned ON', error_message='Error while turning heating ON')
def flip_heating_on():
    heater_button.description = "Heating ON"
    heater_button.button_style = 'success'
    heater_button.icon = 'check'
    heater_button.value = 1
    plate.start_heating()

@log(success_message='Heating Turned OFF', error_message='Error while turning heating OFF')
def flip_heating_off():
    heater_button.description = "Heating OFF"
    heater_button.button_style = 'danger'
    heater_button.icon = 'times' 
    heater_button.value = 0
    plate.stop_heating()

@log(success_message='Stirring Turned on', error_message='Error while: Stirring Turned on')
def flip_stirring_on():
    stirrer_button.description = "Stirring ON"
    stirrer_button.button_style = 'success'
    stirrer_button.icon = 'check'
    stirrer_button.value = 1
    plate.start_stirring()
    
@log(success_message='Stirring Turned OFF', error_message='Error while: Stirring Turned OFF')
def flip_stirring_off():
    stirrer_button.description = "Stirring OFF"
    stirrer_button.button_style = 'danger'
    stirrer_button.icon = 'times'
    stirrer_button.value = 0
    plate.stop_stirring()
    
@log(success_message='flip_reload_on', error_message='Error while: flip_reload_on')    
def flip_reload_on():
    global loading_prev
    global number_of_inputs, interval,posibble_save_intervals, autosave_interval,selected_save,default_port, try_all_ports
    reload_prev_button.button_style = 'info' 
    reload_prev_button.icon = 'check'
    loading_prev = reload_prev_button.value
    create_config(number_of_inputs,reload_prev_button.value,interval,autosave_interval,selected_save, default_port, try_all_ports)    
@log(success_message='flip_reload_off', error_message='Error while: flip_reload_off')
def flip_reload_off():
    global loading_prev
    global number_of_inputs, interval,posibble_save_intervals, autosave_interval,selected_save,default_port, try_all_ports
    reload_prev_button.button_style = '' 
    reload_prev_button.icon = 'times' 
    loading_prev = reload_prev_button.value
    create_config(number_of_inputs,reload_prev_button.value,interval,autosave_interval,selected_save, default_port, try_all_ports)

In [15]:
#Turning time intervals to total time and corresponidng time points
def Cumulate_time():
    for i in np.arange(0,len(Set_time_points)):
        if Set_time_points[i] == 0:
            Cumulative__set_time_points.append(Cumulative__set_time_points[-1]+1)
        else:
            Cumulative__set_time_points.append(Cumulative__set_time_points[-1]+Set_time_points[i])

In [16]:
#generating heating points
def Generate_linear_profiles(i):
    #Determining temperature change direction
    if Set_temperature_points[i] > Set_temperature_points[i-1]:
        direction_of_dT = 1
    else:
       direction_of_dT = -1
    #Holding temperature
    if Set_temperature_points[i-1] == Set_temperature_points[i]:
        Generated_time_points.append(Cumulative__set_time_points[i])
        if Set_temperature_points[i+1] > Set_temperature_points[i]:
            Generated_temperature_points.append(Set_temperature_points[i]+1)
        else:
            Generated_temperature_points.append(Set_temperature_points[i]-1)
    #Changing temperature
    else:
        slope = (Set_temperature_points[i]-Set_temperature_points[i-1])/(Cumulative__set_time_points[i]-Cumulative__set_time_points[i-1])
        for every_degree in np.arange(Set_temperature_points[i-1]+(2*direction_of_dT),Set_temperature_points[i]+direction_of_dT,direction_of_dT):
            Generated_temperature_points.append(every_degree)
            Generated_time_points.append(Cumulative__set_time_points[i-1]+(every_degree-(Set_temperature_points[i-1])-direction_of_dT)/slope) 

In [17]:
#generating temperature profiles
def Generate_prefect_profile(i):
    if Set_temperature_points[i-1] == Set_temperature_points[i]:
        Perfect_time_points.append(Cumulative__set_time_points[i])
        Perfect_temperature_points.append(Set_temperature_points[i])
    elif Set_temperature_points[i] > Set_temperature_points[i-1]:
        slope = (Set_temperature_points[i]-Set_temperature_points[i-1])/(Cumulative__set_time_points[i]-Cumulative__set_time_points[i-1])
        for every_degree in np.arange(Set_temperature_points[i-1]+1,Set_temperature_points[i]+1):
            Perfect_temperature_points.append(every_degree)
            Perfect_time_points.append(Cumulative__set_time_points[i-1]+(every_degree-Set_temperature_points[i-1])/slope)      
    else:
        slope = (Set_temperature_points[i]-Set_temperature_points[i-1])/(Cumulative__set_time_points[i]-Cumulative__set_time_points[i-1])
        for every_degree in np.arange(Set_temperature_points[i-1]-1,Set_temperature_points[i]-1,-1):
            Perfect_temperature_points.append(every_degree)
            Perfect_time_points.append(Cumulative__set_time_points[i-1]+(every_degree-Set_temperature_points[i-1])/slope)


In [18]:
#Update loop
#Update plot and check for other data changes
def update_data():
    #Setup variables and plot
    global Logged_time_points, interval, Logged_temperature_points,Perfect_time_points,Perfect_temperature_points, measurement_date
    global zoomin,zoom, autosave_interval,plate_connected,plate_connected_timeout, just_restarted
    Logged_time_points = []
    measurement_date = []
    measurement_date.append(str(datetime.datetime.now()))
    Logged_temperature_points = []
    Logged_time_points.append(0)
    fig, ax = plt.subplots(figsize=(12, 5.5))
    plt.ioff()
    Starting_time = datetime.datetime.now()
    ax.set_ylabel('T [°C]')
    ax.set_xlabel('time [min]')
    fig.suptitle('Plot of temperature measurments and the planned heating profile \nStarted on: '+str(datetime.datetime.now())[:16], fontsize=14)
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    zoom = 10
    zoom_inner = 10
    zoomin = False
    with plot_output:
        fig.canvas.draw()
    timeout = 0
    update_interval = interval
    logging.info(str(datetime.datetime.now()) + '   ' + 'Starting update thread')
    #update loop, should never stop
    while True:
        timeout += interval
        try:
            if plate.read_device_name() == 'RCT digital':
                plate_connected = True    
        except:
            pass    
        #updating countdown:
        try:
            time_left = [int(s) for s in info_widget.value.replace(':',' ').split() if s.isdigit()]
            time_left1 = np.copy(time_left)
            if time_left:
                if time_left[1] > interval:
                    time_left[1] -= interval
                elif time_left[0] == 0:
                    pass
                else:
                    time_left[0] -= 1
                    time_left[1] += 59-interval
                info_widget.value=info_widget.value.replace(str(time_left1[0]) +":"+ str(time_left1[1]),str(int(time_left[0])) +":"+ str(int(time_left[1])))
        except:
            pass
        #checking heating and stirring and updates buttons and sliders
        try:
            if timeout % 30 == 0 and plate_connected == True:
                stirrer_speed_slider.value = plate.target_stir_rate 
                heater_temp_slider.value = plate.target_temperature
                #if plate.heating != heater_button.value:
                #    if plate.heating:
                #        flip_heating_on()
                #    else:
                #        flip_heating_off()
                #if plate.stirring != stirrer_button.value:    
                #   if plate.stirring:
                #       flip_stirring_on()
                #    else:
                #        flip_stirring_off()
               # if just_restarted:
               #     flip_stirring_on()
               #     flip_heating_on()
               #     just_restarted = False
        except:
            logging.error(str(datetime.datetime.now()) + '   ' + 'Error while checking heating and stirring and updates buttons and sliders')
        
        try:
            Logged_temperature_points.append(plate.probe_temperature)
        except:
            Logged_temperature_points.append(10)
            plate_connected = False
            logging.error(str(datetime.datetime.now()) + '   ' + 'Error while updating probe temperature, trying to reconnect to plate')
            try:
                reconnect_to_plate()
                plate_connected = True
            except Exception as eee:
                logging.error(str(datetime.datetime.now()) + '   ' + 'Failed to estabilish connection to plate'+ str(eee))
        #autosave, overwrites previous!
        if timeout % autosave_interval == 0 and plate_connected == True and plate_connected_timeout == True:
            try:
                pass
                #read_data_from_client()
            except:
                pass
            try:
                autosave()
            except Exception as e:
                logging.error('Autosave failed'+str(e)+ str(datetime.datetime.now()))
                pass
        if update_interval>0:
            time.sleep(update_interval)
        update_interval = interval-((datetime.datetime.now()-Starting_time).total_seconds() -timeout)
    
        #if some error occurs, remove the last data point
        if not len(Logged_time_points) == len(Logged_temperature_points):
            if len(Logged_time_points) == max(len(Logged_temperature_points),len(Logged_time_points)):
                Logged_time_points.pop()    
            else:
                Logged_temperature_points.pop()
        else:
            pass
        #updating graph
        with plot_output:
            try:
                ax.clear()
                if Generated_time_points == []:
                    ax.plot([x/60 for x in Generated_time_points],Generated_temperature_points, label = 'Planned heating points', linestyle ='',marker ='o')
                    ax.plot([x/60 for x in Perfect_time_points],Perfect_temperature_points, label = 'Planned temperatures')
                    ax.plot([x/60 for x in Logged_time_points], Logged_temperature_points, label='Measured temperature',linewidth=3.0)
                    ax.set_xlim(xmin=0)
                    try:
                        if zoomin:
                            zoom_inner = zoom + 1 - 1
                            ax.set_xlim(xmin=max(0,(Logged_time_points[-1]/60-zoom_inner)), xmax=(Logged_time_points[-1]/60+zoom_inner))
                            ax.set_ylim(ymin=max(15,(Logged_temperature_points[-1]-zoom_inner/5)), ymax=(Logged_temperature_points[-1]+zoom_inner/5))
                    except:
                        pass
                    ax.set_ylabel('T [°C]')
                    ax.set_xlabel('time [min]')    
                else: 
                    if max(Generated_time_points)/60 < 180: 
                        ax.plot([x/60 for x in Generated_time_points],Generated_temperature_points, label = 'Planned heating points', linestyle ='',marker ='o')
                        ax.plot([x/60 for x in Perfect_time_points],Perfect_temperature_points, label = 'Planned temperatures')
                        ax.plot([x/60 for x in Logged_time_points], Logged_temperature_points, label='Measured temperature',linewidth=3.0)
                        ax.set_xlim(xmin=0)
                        try:
                            if zoomin:
                                zoom_inner = zoom + 1 - 1
                                ax.set_xlim(xmin=max(0,(Logged_time_points[-1]/60-zoom_inner)), xmax=(Logged_time_points[-1]/60+zoom_inner))
                                ax.set_ylim(ymin=max(15,(Logged_temperature_points[-1]-zoom_inner/5)), ymax=(Logged_temperature_points[-1]+zoom_innerm/5))
                        except:
                            pass
                        ax.set_ylabel('T [°C]')
                        ax.set_xlabel('time [min]')
                    else:
                        ax.plot([x/60/60 for x in Generated_time_points],Generated_temperature_points, label = 'Planned heating points', linestyle ='',marker ='o')
                        ax.plot([x/60/60 for x in Perfect_time_points],Perfect_temperature_points, label = 'Planned temperatures')
                        ax.plot([x/60/60 for x in Logged_time_points], Logged_temperature_points, label='Measured temperature',linewidth=3.0)
                        ax.set_xlim(xmin=0)
                        try:
                            if zoomin:
                                zoom_inner = zoom + 1 - 1
                                ax.set_xlim(xmin=max(0,(Logged_time_points[-1]/60/60-zoom_inner/60)), xmax=(Logged_time_points[-1]/60/60+zoom_inner/60))
                                ax.set_ylim(ymin=max(15,(Logged_temperature_points[-1]-zoom_inner/5)), ymax=(Logged_temperature_points[-1]+zoom_inner/5))
                        except:
                            pass
                        ax.set_ylabel('T [°C]')
                        ax.set_xlabel('time [hours]')    
                #
                ax.legend()
                fig.canvas.draw()
            except Exception as err:
                logging.error(str(datetime.datetime.now()) + '     ' + 'Error while updating graph'+ str(err))
            


        Logged_time_points.append(Logged_time_points[-1] + interval)
        measurement_date.append(str(datetime.datetime.now()))

In [19]:
#Control loop
def Start_Control():
    logging.info("trying to start controll thread")
    global Logged_time_points, interval, current_point_data, loading_prev
    global Generated_time_points,Perfect_time_points, Start_point,restart
    global stop_heating_after_program, stop_stirring_after_program, disconnect_after_program
    global plate_connected
    try:
        whole_control_starting_time=datetime.datetime.now()
        starting_point = int(current_point_data[0])
        if loading_prev:
            starting_point = Start_point
            current_point_data = [int(starting_point),"Waiting"]
    except Exception as e:
        logging.error(str(datetime.datetime.now())+ '     ' + 'Error occured while initiating control'+ str(e) )    
    def sleep_with_event_check(duration, check_interval=1):
        try:
            elapsed = 0
            timestep = 0
            control_starting_time = datetime.datetime.now()
            updated_control_interval=check_interval
            while elapsed < duration:
                timestep += 1
                #info_widget.value = str((whole_control_starting_time-datetime.datetime.now()).total_seconds())
                if updated_control_interval < 0:
                    updated_control_interval = 0
                time.sleep(min(updated_control_interval,duration-elapsed))
                elapsed += updated_control_interval
                
                updated_control_interval = check_interval-((datetime.datetime.now()-control_starting_time).total_seconds() -timestep)
                if stop_control_event.is_set():
                    return
        except Exception as ee:
            logging.error(str(datetime.datetime.now()) + '   ' + 'Error occured during waiting, skipping wait time...'+ str(ee))
    try:
        if not loading_prev or restart:
            Generated_time_points2 = np.copy(Generated_time_points)
            Generated_time_points = [x+Logged_time_points[-1] for x in Generated_time_points2]
            Perfect_time_points2 = np.copy(Perfect_time_points)
            Perfect_time_points = [x+Logged_time_points[-1] for x in Perfect_time_points2]
        else:
            Generated_time_points2 = np.copy(Generated_time_points)
            Perfect_time_points2 = np.copy(Perfect_time_points)
        plate.target_temperature = Generated_temperature_points[int(starting_point)]    
        plate.start_heating()
        flip_heating_on()
        logging.info(str(datetime.datetime.now()) + '   ' + 'Starting control thread')
        loading_prev = False        
        restart = False
        while True:
            restart_for_loop = False
            for point in np.arange(int(starting_point),len(Generated_temperature_points)):
                current_point_data[0] = point
                if point == len(Generated_temperature_points):
                    Start_control_button_clicked("b") 
                info_widget.value = "Target temperature:" + str(Generated_temperature_points[point])
                try:
                    try:
                        plate.target_temperature = Generated_temperature_points[point]
                        logging.info(str(datetime.datetime.now()) + '   ' + 'Target temperature for   ' + str(Generated_temperature_points[point]) +'  set')
                    except:
                        logging.error('Error occured while setting target temp' + str(datetime.datetime.now()))
                    
                    if point > 0 and point < len(Generated_temperature_points):
                        wait = True
                        timeout_counter = 0
                        current_point_data[1]="Waiting"
                        while wait == True:
                            try:
                                sleep_with_event_check(interval)
                                timeout_counter += 1
                                try:
                                    current_temp = plate.probe_temperature
                                    current_target = plate.target_temperature
                                except:
                                    current_temp = 20
                                    current_target =21
                                    plate_connected = False
                                    logging.error('Error while reading probe temperature and target, trying to reconnect'+ str(datetime.datetime.now()))
                                    try:
                                        reconnect_to_plate()
                                        plate_connected = True
                                        current_temp = plate.probe_temperature
                                        current_target = plate.target_temperature
                                    except Exception as eee:
                                        logging.info(str(datetime.datetime.now()) + '   ' + 'Failed to estabilish connection to plate'+ str(eee))
                                info_widget.value = "Target temperature:" + str(Generated_temperature_points[point]) + " is lower than probe temp.:" + str(current_temp)
                                if current_target <  Generated_temperature_points[point-1]:
                                     if current_target >= current_temp:
                                        wait = False
                                        info_widget.value = "Goal Temperature reached"
                                        logging.info(str(datetime.datetime.now()) + '   ' + 'Goal Temperature of   '+str(Generated_temperature_points[point]) +'  first reached')
                                else:
                                    if current_target <= current_temp:
                                        wait = False
                                        info_widget.value = "Goal Temperature reached"
                                        logging.info(str(datetime.datetime.now()) + '   ' + 'Goal Temperature of   '+str(Generated_temperature_points[point]) +'  first reached')
                                if timeout_counter >= 1800:
                                    wait = False 
                                    info_widget.value = "Goal Temperature wasn't reached within timeout limits"
                                    logging.error('Goal Temperature wasnt reached within timeout limit     ' + str(datetime.datetime.now()))
                                if stop_control_event.is_set():
                                    break
                            except:
                                time.sleep(10)
                                logging.error('Error while waiting to reach target temperature'+ str(datetime.datetime.now()))
                    #when reaching the last temperature point turn off controll                          
              
                                    
                    while stop_control_event.is_set():
                        Start_control_button.disabled = False
                        info_widget.value = "Control paused"
                          
                        time.sleep(interval)
                        if len(Generated_time_points) == len(Generated_time_points2):
                            Generated_time_points = [x+Logged_time_points[-1] for x in Generated_time_points2]
                  
                        else:
                            Generated_time_points2 =  np.copy(Generated_time_points)
                            restart_for_loop = True
                            starting_point = 0
                        if len(Perfect_time_points) == len(Perfect_time_points2):
                            Perfect_time_points = [x+Logged_time_points[-1] for x in Perfect_time_points2]
                        else:
                            Perfect_time_points2 =  np.copy(Perfect_time_points)
                    if point != len(Generated_temperature_points):
                        try:
                            info_widget.value = "Goal Temperature reached, waiting for next heating step in: " + str(math.floor((Generated_time_points[point+1]-Generated_time_points[point])/60)) +":"+str(int((((Generated_time_points[point+1]-Generated_time_points[point])/60)-math.floor((Generated_time_points[point+1]-Generated_time_points[point])/60))*60)) + " min:s"                    
                        except:
                            pass
                    if restart_for_loop:
                        break
                    current_point_data[1]="Holding"
                    logging.info(str(datetime.datetime.now()) + '   ' + 'Starting holding temperature')
                    if point != len(Generated_temperature_points):
                        sleep_with_event_check(Generated_time_points[point+1]-Generated_time_points[point])
                    logging.info(str(datetime.datetime.now()) + '   ' + 'Finished holding temperature')
                except Exception as ee:
                    logging.error(str(datetime.datetime.now()) + 'Error During controll'+str(ee))
                    time.sleep(10)
                    
            if not restart_for_loop:
                info_widget.value = "Control finished"
                break
        if stop_heating_after_program:
            plate.stop_heating()
        if stop_stirring_after_program:
            pass
        if disconnect_after_program:
            pass
        info_widget.value = "Control finished"
        logging.error(str(datetime.datetime.now()) + '     '+'Control finished')
        logging.error(str(datetime.datetime.now()) + '     '+'Trying to restart control')
        #plate.stop_heating()
        #plate.disconnect()
        time.sleep(10)
        restart = True
        plate_connected = False
        try:
            if not plate.read_device_name() == 'RCT digital':
                reconnect_to_plate()
        except:        
            try:
                reconnect_to_plate()
            except Exception as eee:
                logging.info(str(datetime.datetime.now()) + '   ' + 'Failed to estabilish connection to plate'+ str(eee))
        #Start_control_button_clicked("b")
        #starting_again = False

        control_thread = threading.Thread(target=Start_Control,daemon='True')
        control_thread.start()
        load_previous("restart")
        time.sleep(2)
        Start_control_button_clicked("b")
        
    except Exception as e:
        logging.error('Error occured, heating shutdown, plate disconnected, trying to restart control     ' + str(datetime.datetime.now()))
        try:
            logging.error(str(e))
            info_widget.value = str(e)
        except:
            pass
        #plate.stop_heating()
        #plate.disconnect()
        
        time.sleep(10)
        restart = True
        try:
            reconnect_to_plate()
        except Exception as eee:
            logging.info(str(datetime.datetime.now()) + '   ' + 'Failed to estabilish connection to plate'+ str(eee))
        #Start_control_button_clicked("b")
        #starting_again = False
        
        control_thread = threading.Thread(target=Start_Control,daemon='True')
        control_thread.start() 
        load_previous("restart")
        time.sleep(2)
        Start_control_button_clicked("b")

In [20]:
#Load previous save file
selected_save =''
def load_previous(selected=selected_save):
    try:
        global selected_save
        try:
            if selected == "restart" and os.path.exists('autosaves/Current_state.csv'):
                reload_data_from_previous_measurment(location_prev_state='autosaves/Current_state.csv')
            elif selected == 'Last save' or selected == "restart":
                reload_data_from_previous_measurment()               
            else:
                reload_data_from_previous_measurment(location_prev_state='autosaves/previous_state'+str(selected)+'.csv') 

        except:
            reload_data_from_previous_measurment()
        Start_control_button_clicked("b")
    except:
        pass

In [21]:
#recconnect to plate, calls connect_to_device function, and if it succeds sets plate_connected = True
def reconnect_to_plate():
    global plate_connected
    connect_to_device()
    plate_connected = True
    connection_timeout_thread = threading.Thread(target=plate_connection_timeout,daemon='True')
    connection_timeout_thread.start()

In [22]:
#disables autosave for autosave_interval + 5 seconds, to prevent errors while reloading save after an unexpected error
def plate_connection_timeout():
    global plate_connected_timeout,autosave_interval,just_restarted
    plate_connected_timeout = False
    time.sleep(autosave_interval + 5)
    plate_connected_timeout= True
    just_restarted = True

In [23]:
#settings menu
@log(success_message='Opening settings...', error_message='Opening settings failed')
def open_settings_window():
    #Change No of Inputs
    def on_combobox_1_change(event):
        global number_of_inputs, interval, autosave_interval,selected_save,default_port, try_all_ports
        try:
            number_of_inputs = int(general_option_1.get())
            create_config(int(number_of_inputs),reload_prev_button.value,interval,autosave_interval,selected_save,  default_port, try_all_ports)
            logging.info(str(datetime.datetime.now()) + '   ' + 'Changed number of inputs to:' + str(number_of_inputs))
        except:
            pass
    #Change logging interval
    def on_combobox_2_change(event):
        global number_of_inputs, interval,posibble_save_intervals, autosave_interval,selected_save,default_port, try_all_ports
        try:
            interval = int(general_option_2.get())
            create_config(number_of_inputs,reload_prev_button.value,int(interval),autosave_interval,selected_save, default_port, try_all_ports)
            #This line is supposed to make sure that the autsave interval is divisible by the logging interval, but it doesnt work, IDK
            posibble_save_intervals = [1*interval,10*interval,15*interval,30*interval,60*interval,120*interval,240*interval,480*interval]
            logging.info(str(datetime.datetime.now()) + '   ' + 'Changed logging interval to: [s]' + str(interval))
        except:
            pass
    #Change autosave interval
    def on_combobox_3_change(event):
        global number_of_inputs, interval,autosave_interval, selected_save,default_port, try_all_ports
        try:
            autosave_interval = int(general_option_3.get())
            create_config(number_of_inputs,reload_prev_button.value,int(interval),int(autosave_interval),selected_save, default_port, try_all_ports)
            logging.info(str(datetime.datetime.now()) + '   ' + 'Changed autosave interval to: [s]' + str(interval))
        except:
            pass
    #select save file
    def on_combobox_4_change(event):
        global number_of_inputs, interval,autosave_interval, selected_save,default_port, try_all_ports
        try:
            selected_save = advanced_option_2.get()
            selected_save =(
            datetime.datetime.strptime(selected_save, "%Y/%m/%d %H:%M").strftime("%Y%m%d%H%M")
            if len(selected_save) == 16 and selected_save[4] == '/' and selected_save[7] == '/' and selected_save[13] == ':'
            else selected_save
            )
            create_config(number_of_inputs,reload_prev_button.value,int(interval),int(autosave_interval),selected_save, default_port, try_all_ports)
            logging.info(str(datetime.datetime.now()) + '   ' + 'Changed previous save to' + str(selected_save))
        except:
            pass
    #Change default port
    def on_default_port_change(event):
        global number_of_inputs, interval,autosave_interval, selected_save,default_port, try_all_ports
        try:
            default_port = str(general_option_4.get())
            create_config(number_of_inputs,reload_prev_button.value,int(interval),int(autosave_interval),selected_save, default_port, try_all_ports)
            logging.info(str(datetime.datetime.now()) + '   ' + 'Changed DEFAULT PORT save to' + str(default_port))
        except:
            pass
    #Change wether to try all available serial ports
    def on_try_all_ports_change(event):
        global number_of_inputs, interval,autosave_interval, selected_save,default_port, try_all_ports
        try:
            if str(general_option_5.get()) == 'Yes':
                try_all_ports = True
            else:
                try_all_ports = False
            create_config(number_of_inputs,reload_prev_button.value,int(interval),int(autosave_interval),selected_save, default_port, try_all_ports)
            logging.info(str(datetime.datetime.now()) + '   ' + 'Changed try all ports to: ' + str(try_all_ports)) 
        except:
            pass  
    def read_log_file():
        try:
            infile = 'logs/logfile' +  date_of_measurment + ".log"
            #error_messages = ''
            keep_phrases = ["ERROR:"]
            listbox.delete(0,listbox.size())
            with open(infile) as f:
                f = f.readlines()
            
            for line in f:
                for phrase in keep_phrases:
                    if phrase in line:
                        #error_messages += (str(line[11:]))
                        listbox.insert(tk.END, (str(line[11:])))
                        break
            #advanced_label_4.config(text = error_messages)
        except:
            pass
    def debug_mode_switch():
        global number_of_inputs, interval,autosave_interval, selected_save,default_port, try_all_ports, debug_mode
        try:
            debug_mode = var1.get()
            create_config(number_of_inputs,reload_prev_button.value,int(interval),int(autosave_interval),selected_save, default_port, try_all_ports,debug_mode)
            logging.info(str(datetime.datetime.now()) + '   ' + 'Changed try all ports to: ' + str(try_all_ports)) 
        except:
            pass

    global number_of_inputs, interval, autosave_interval,posibble_save_intervals,selected_save, DeviceID,debug_mode
    possible_no_of_inputs = [3,4,5,6,7,8,9,10,11,12,13,14,15]
    possible_length_of_intervals = [1,2,3,5,6,10,15,30,60,120,240,480]
    posibble_save_intervals = [1,10,15,30,60,120,240,480]
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    settings_window = tk.Toplevel()
    settings_window.title("Settings")
    settings_window.geometry("650x600")
    settings_window.protocol("WM_DELETE_WINDOW", lambda: on_close(settings_window, root))
    # Create a notebook (tab container)
    tab_control = ttk.Notebook(settings_window)

    # Create the tabs
    general_tab = ttk.Frame(tab_control)
    advanced_tab = ttk.Frame(tab_control)

    tab_control.add(general_tab, text='General')
    tab_control.add(advanced_tab, text='Advanced')

    tab_control.pack(expand=1, fill="both")

    # Populate the General tab with some settings
    general_label = ttk.Label(general_tab, text="General Settings", padding=10)
    general_label.pack()
    
    general_label_1 = ttk.Label(general_tab, text="Select number of input fields (Temperature points)")
    general_option_1 = ttk.Combobox(general_tab, values=possible_no_of_inputs, state='readonly')
    general_option_1.bind("<<ComboboxSelected>>", on_combobox_1_change)    
    # Set a default value (optional)
    general_option_1.set(number_of_inputs)

    general_label_1.pack()
    general_option_1.pack()
    general_label_2 = ttk.Label(general_tab, text="Select length of update interval [s]")
    general_option_2 = ttk.Combobox(general_tab, values=possible_length_of_intervals, state='readonly')
    general_option_2.bind("<<ComboboxSelected>>", on_combobox_2_change)    
    # Set a default value (optional)
    general_option_2.set(interval)
    general_label_2.pack()
    general_option_2.pack()
    
    general_label_3 = ttk.Label(general_tab, text="Select length of autosave interval [s]")
    general_option_3 = ttk.Combobox(general_tab, values=possible_length_of_intervals, state='readonly')
    general_option_3.bind("<<ComboboxSelected>>", on_combobox_3_change)    
    # Set a default value (optional)
    general_option_3.set(autosave_interval)
    general_label_3.pack()
    general_option_3.pack()
    ##Defport
    general_label_4 = ttk.Label(general_tab, text="Select default port")
    general_option_4 = ttk.Combobox(general_tab, values=DeviceID, state='normal')
    general_option_4.bind("<<ComboboxSelected>>", on_default_port_change)    
    # Set a default value (optional)
    general_option_4.set(default_port)
    general_label_4.pack()
    general_option_4.pack()

    ##Defport
    general_label_5_0 = ttk.Label(general_tab, text="Do you want to try all connected ports?", padding=10)
    general_label_5_1 = ttk.Label(general_tab, text="Set this to YES to try all connected serial ports to the computer")
    general_label_5_2 = ttk.Label(general_tab, text="(this could potentially cause problems with other connected devices)")
    general_label_5_3 = ttk.Label(general_tab, text="Set this to NO to only try to connect to the default port.")
    general_option_5 = ttk.Combobox(general_tab, values=['YES','NO'], state='normal')
    general_option_5.bind("<<ComboboxSelected>>", on_try_all_ports_change)    
    # Set a default value (optional)
    if try_all_ports:
        general_option_5.set('YES')
    else:
        general_option_5.set('NO') 
    general_label_5_0.pack()
    general_label_5_1.pack()
    general_label_5_2.pack()
    general_label_5_3.pack()
    general_option_5.pack()
    # Populate the Advanced tab with some settings
    advanced_label = ttk.Label(advanced_tab, text="Advanced Settings", padding=10)
    advanced_label.pack()

    #advanced_option_1 = ttk.Checkbutton(advanced_tab, text="Reload specific save file")
    #advanced_option_1.pack()
    possible_savefiles = []
    possible_savefiles = os.listdir('autosaves/')
    possible_savefiles = [
        cleaned for x in possible_savefiles
        if (cleaned := ''.join([char for char in x if not char.isalpha() and char not in '._' ]))
    ]
    possible_savefiles.append('Last save')
    possible_savefiles = [
        (datetime.datetime.strptime(date, "%Y%m%d%H%M").strftime("%Y/%m/%d %H:%M") if len(date) == 12 else date)
        if len(date) == 12 and date.isdigit()
        else date
        for date in possible_savefiles
    ]      
    advanced_label_2 = ttk.Label(advanced_tab, text="Select save to reload")
    advanced_label_3 = ttk.Label(advanced_tab, text='Select the date of the save file you wish to open, then close this window, turn on reload previous save and refresh the page, to reload last save select "Last save"',wraplength=300)
    advanced_option_2 = ttk.Combobox(advanced_tab, values=possible_savefiles, state='readonly')
    advanced_option_2.bind("<<ComboboxSelected>>", on_combobox_4_change)    
    # Set a default value (optional)
    advanced_option_2.set(selected_save)
    advanced_label_2.pack()
    advanced_option_2.pack()
    advanced_label_3.pack()
    var1 = tk.BooleanVar()
    var1.set(debug_mode)
    c1 = tk.Checkbutton(advanced_tab, text='Debug mode:\n (If turned on defaults to a mock heating pad when none is connected,\n turn this option OFF for production use)',variable = var1, onvalue=True, offvalue=False, command=debug_mode_switch)
   
    c1.pack()
    #advanced_label_4 = ttk.Label(advanced_tab, text="Display errors in logfile")
    advanced_option_4 = ttk.Button(advanced_tab,  text="Display errors in logfile", command=read_log_file)

    #advanced_label_4 = ttk.Label(advanced_tab, text="Display errors in logfile")
    # Set a default value (optional)
    advanced_option_4.pack()
    #advanced_label_4.pack()
    listbox = tk.Listbox(advanced_tab)
    listbox.pack(side="left", fill="both", expand=1)
    
    scrollbar = ttk.Scrollbar(advanced_tab, orient="vertical", command=listbox.yview)
    scrollbar.pack(fill="y", expand=1)
    
    listbox.configure(yscrollcommand=scrollbar.set)

    settings_window.mainloop()
   
def on_close(window, root):
    window.destroy()  # Destroy the Toplevel window
    root.quit()       # Ensure the Tkinter main loop exits


In [24]:
#Buttons and sliders
reload_prev_button = widgets.ToggleButton(
    value=False,
    description='Reload previous save',
    button_style='',
    tooltip='Try to reload previous save on next start',
    icon='times'
)
stirrer_button = widgets.ToggleButton(
    value=False,
    description='Stirring OFF',
    button_style='danger',
    tooltip='Stirring',
    icon='times'
)
stirrer_speed_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=1500,
    step=10,
    description='RPM:',
    readout=True,
    readout_format='d'
)
zoom_in_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=300,
    step=5,
    description='Zoom:',
    readout=False,
    readout_format='d'
)
heater_temp_slider = widgets.IntSlider(
    value=0,
    min=15,
    max=310,
    step=1,
    description='°C',
    disabled=True,
    readout=True,
    readout_format='d'
)
heater_button = widgets.ToggleButton(
    value=False,
    description='Heating OFF',
    disabled=True,
    button_style='danger', 
    tooltip='heating',
    icon='times' 
)
enable_heater_checkbox = widgets.Checkbox(
    value=False,
    description='Enable heater control',
    indent=False
)
reset_graph_button = widgets.Button(description="Reset temperature profile to 0 postion")
settings_button = widgets.Button(description="Settings")
Connect_to_device_button = widgets.Button(description="Connect to heating pad")
generate_profile_button = widgets.Button(description="Generate temperature profile")
Start_control_button = widgets.Button(description="Start Control", disabled = True)
save_button = widgets.Button(description="Save")
output = widgets.Output()
reset_graph_button.layout.width = '400px'
Connect_to_device_button.layout.width = '300px'  # Set width to 300 pixels
generate_profile_button.layout.width = '300px'  # Set width to 300 pixels
info_widget=widgets.HTML(value="<b>Info</b>")
zoom_in_slider.layout.width = '400px'

In [25]:
#Button and slider functions
@log(success_message='Graph reset', error_message='Error after pressing Graph reset button')
def reset_graph(b):
    global Generated_time_points,Perfect_time_points
    Generated_time_points2 = np.copy(Generated_time_points)
    Generated_time_points = [x-Generated_time_points2[0] for x in Generated_time_points2]
    Perfect_time_points2 = np.copy(Perfect_time_points)
    Perfect_time_points = [x-Perfect_time_points2[0] for x in Perfect_time_points2]
def on_reload_prev_button_clicked(b):
    if reload_prev_button.value:
        flip_reload_on()
        logging.info('a')
    else:
        flip_reload_off()
        logging.info('b')
@log(success_message='Control button pressed', error_message='Error after pressing control button')
def Start_control_button_clicked(b):
    generate_profile_button.disabled = True
    Start_control_button.disabled = False
    Start_control_button.button_style ='success'
    Start_control_button.icon = 'check'
    for i in np.arange(0,number_of_inputs):       
        Temp_box[i].disabled = True
        Time_to_complete_box[i].disabled = True
        RampRate_box[i].disabled = True
        Time_to_hold_box[i].disabled = True
    if Start_control_button.description == "Start Control":
        Start_control_button.description = "Controlling temperature"
        global control_thread,starting_again
        stop_control_event.clear()
        if not starting_again:
            control_thread = threading.Thread(target=Start_Control,daemon='True')
            control_thread.start()
    else:
        starting_again = True
        Start_control_button.disabled = True
        Time_to_hold_box[0].disabled = False
        Start_control_button.description = "Start Control"
        stop_control_event.set()
        generate_profile_button.disabled = False
        Start_control_button.button_style ='danger'
        Start_control_button.icon = 'cross'
        for i in np.arange(1,number_of_inputs):       
            Temp_box[i].disabled = False
            Time_to_complete_box[i].disabled =False
            RampRate_box[i].disabled = False
            Time_to_hold_box[i].disabled = False

@log(success_message='heater control enabled/disabled by user', error_message='Error after heater control enabled/disabled by user')
def on_heater_enabler_clicked(b):
    with output:
        if enable_heater_checkbox.value:
            heater_button.disabled = False
            heater_temp_slider.disabled = False
        else:
            heater_button.disabled = True
            heater_temp_slider.disabled = True
            
def on_stirrer_button_clicked(b):
    with output:
        if stirrer_button.value:
            flip_stirring_on()
        else:
            flip_stirring_off()
            
def on_heater_button_clicked(b):
    with output:
        if heater_button.value:
            flip_heating_on()
        else:
            flip_heating_off()  
            
def on_heater_slider_changed(b):
    with output:
        try:
            plate.target_temperature = heater_temp_slider.value
            logging.info(str(datetime.datetime.now()) + '   ' + 'Heating temp changed to  '+str(heater_temp_slider.value))
        except:
            logging.error(str(datetime.datetime.now()) + '   ' + 'Failed to change Heating temp to  '+str(heater_temp_slider.value))
def on_stirrer_slider_changed(b):
    with output:
        try:
            plate.set_speed_value(stirrer_speed_slider.value)
            logging.info(str(datetime.datetime.now()) + '   ' + 'Stirring speed changed to  ' + str(stirrer_speed_slider.value))
        except:
            logging.error(str(datetime.datetime.now()) + '   ' + 'Failed to change stirring speed to:  ' + str(stirrer_speed_slider.value))    

def on_Connect_to_device_button_clicked(b):
    try:
        global selected_port,plate
        plate = MagneticStirrer(device_port=selected_port)
        Connect_to_device_button.button_style ='success'
        Connect_to_device_button.disabled = True
    except:
        connect_to_device()
        #plate = MockMagneticStirrer()
        #Connect_to_device_button.button_style ='danger'


@log(success_message='Temperature profile generated succesfully', error_message='Temperature profile generation error')
def on_generate_profile_button_clicked(b):
    temperatures = []
    times =[]
    Temp_box[0].value =  math.ceil(plate.probe_temperature)
    Cell_No = 0
    if Temp_box[Cell_No+1].value < Temp_box[Cell_No].value:
        Time_to_complete_box[Cell_No+1].value = (Temp_box[Cell_No].value-Temp_box[Cell_No+1].value)/(RampRate_box[Cell_No+1].value/60)
    else:
        Time_to_complete_box[Cell_No+1].value = (Temp_box[Cell_No+1].value-Temp_box[Cell_No].value)/(RampRate_box[Cell_No+1].value/60)  
    global Set_time_points
    Set_time_points = []
    Set_time_points.append(0)
    global Set_temperature_points,  Generated_temperature_points,  Generated_time_points, Perfect_temperature_points
    global Cumulative__set_time_points,Perfect_time_points
    Cumulative__set_time_points = []
    Set_temperature_points = []
    Generated_temperature_points = []
    Generated_time_points = []
    Cumulative__set_time_points.append(0)
    for i in np.arange(0,number_of_inputs):       
        temperatures.append(Temp_box[i].value)
        temperatures.append(Temp_box[i].value)
        if i != 0:
            times.append(Time_to_complete_box[i].value*60)
        times.append(Time_to_hold_box[i].value*60)
    Set_temperature_points = temperatures 
    Set_time_points = times
    Cumulate_time()
    #generate heating points
    Generated_temperature_points.append(Temp_box[0].value)
    Generated_time_points.append(0)
    for i in np.arange(1,len(Set_temperature_points)-1):
        Generate_linear_profiles(i)
    #generate ideal profile
    Perfect_temperature_points = []
    Perfect_time_points = []
    Perfect_temperature_points.append(Temp_box[0].value)
    Perfect_time_points.append(0)
    for i in np.arange(1,len(Set_temperature_points)):
        Generate_prefect_profile(i) 
    #print( Generated_time_points)
    #print( Generated_temperature_points)
    Start_control_button.disabled = False
    Start_control_button.button_style ='danger'
 
def save_button_clicked(b):
    try:
        # Create a root window and hide it
        root = tk.Tk()
        root.withdraw()
        # Open the "Save As" dialog
        file_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                                 filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if file_path:
            autosave(file_path)
            logging.info(f"Data saved to {file_path}")
        else:
            logging.info("Save operation cancelled.")
    except:
        logging.error(str(datetime.datetime.now()) + '   ' + 'Save error')       

def settings_button_clicked(b):
    open_settings_window()

@log(success_message='Zoom changed', error_message='Error changing zoom')
def on_zoom_in_changed(b):
    global zoomin,zoom
    zoomin = True
    if zoom_in_slider.value == 0:
        zoomin = False
    else:
        zoom = zoom_in_slider.max - zoom_in_slider.value + 5



In [26]:
#setting up global variables
plate_connected = False
just_restarted = False
plate_connected_timeout = True
starting_again = False
Generated_temperature_points = []
Logged_time_points = []
Perfect_time_points =  []
Perfect_temperature_points = []
Set_temperature_points = []
current_point_data = [0,"Waiting"]
Cumulative__set_time_points = []
Generated_time_points = []
Set_time_points = []
stop_event = threading.Event()
stop_control_event = threading.Event()
pause_event = threading.Event()
number_of_inputs = 6 #No of input fields
interval = 1    #logging interval in seconds
autosave_interval = 30 #Autosave interval in seconds
posibble_save_intervals = [1,10,15,30,60,120,240,480]
loading_prev = True
selected_save = 'Last_save'
connected = False
try_all_ports = True
default_port = 'COM3'
stop_heating_after_program = False
stop_stirring_after_program = False
disconnect_after_program = False
debug_mode = False
restart = False

In [27]:
#read config files and generate it if neccesary
try:
    config_data = read_config()
except:
    logging.error(str(datetime.datetime.now()) + '   ' + 'error loading config file, defaulting to defaults...')
    create_config()
    config_data = read_config()
try:
    interval = float(config_data["logging_interval"])
    autosave_interval = int(config_data["save_interval"])
    selected_save = (config_data["selected_save"])
    loading_prev = config_data["load_previous"]
    default_port = config_data["default_port"]
    try_all_ports = config_data["try_all_ports"]
    debug_mode = config_data["debug_mode"]
    reload_prev_button.value = loading_prev
    if loading_prev:
        reload_prev_button.button_style = 'info' 
        reload_prev_button.icon = 'check' 
    number_of_inputs = int(config_data["No_of_boxes"])
    logging.info(str(datetime.datetime.now()) + '   ' + 'Succesfully loaded config file'+str(config_data))
except:
    logging.error(str(datetime.datetime.now()) + '   ' + 'error loading config file, defaulting to defaults...')

In [28]:
#Connection to device
def connect_to_device():
    global connected,DeviceID, selected_port,plate,try_all_ports,default_port,debug_mode,plate_connected
    if not try_all_ports:
        try:
            try:
                plate.disconnect()
            except:
                pass
            DeviceID = [comport.device for comport in serial.tools.list_ports.comports()]
            port = default_port
            plate = MagneticStirrer(device_port=port)
            if plate.read_device_name() == 'RCT digital':
                logging.info(str(datetime.datetime.now()) + '   ' + 'plate connected succesfully, to default port')
                Connect_to_device_button.button_style ='success'
                Connect_to_device_button.disabled = True
                connected = True
                selected_port = str(port)
                plate_connected = True
                try:
                    plate.safe_disconnect = False
                except:
                    try:
                        plate.safe_disconnect(False)
                    except:
                        logging.info(str(datetime.datetime.now()) + '   ' + 'Failed to turn safe disconnect off')  
            else:
                if debug_mode==True:
                    plate = MockMagneticStirrer()
                    plate_connected = True
                Connect_to_device_button.button_style ='danger'
                logging.error(str(datetime.datetime.now()) + '   ' + 'plate connection error, to default port')                
        except Exception as e:
            if debug_mode==True:
                plate = MockMagneticStirrer()
                plate_connected = True
            Connect_to_device_button.button_style ='danger'
            logging.error(str(datetime.datetime.now()) + '   ' + 'plate connection error, to default port'+str(e))
    else:
        try:
            selected_port = ''
            #List all available ports
            DeviceID = [comport.device for comport in serial.tools.list_ports.comports()]
            #move default port to front of the list
            try:
                if str(default_port) in DeviceID:
                    DeviceID.remove(str(default_port))
                    DeviceID.insert(0,str(default_port))
            except:
                DeviceID = [comport.device for comport in serial.tools.list_ports.comports()]
            try:
                plate.disconnect()
            except:
                pass
            for ports in DeviceID:
                try:
                    plate = MagneticStirrer(device_port=ports)
                    if plate.read_device_name() == 'RCT digital':
                        selected_port = str(ports)
                        connected = True
                        plate_connected = True
                        try:
                            plate.safe_disconnect = False
                        except:
                            try:
                                plate.safe_disconnect(False)
                            except:
                                logging.info(str(datetime.datetime.now()) + '   ' + 'Failed to turn safe disconnect off')  
                        break
                except:
                    connected = False
                    
            if connected:
                #plate = MagneticStirrer(device_port=selected_port)
                logging.info(str(datetime.datetime.now()) + '   ' + 'plate connected succesfully, while trying all ports')
                Connect_to_device_button.button_style ='success'
                Connect_to_device_button.disabled = True
            else:
                if debug_mode==True:
                    plate = MockMagneticStirrer()
                    plate_connected = True
                Connect_to_device_button.button_style ='danger'
                logging.error(str(datetime.datetime.now()) + '   ' + 'plate connection error, while trying all ports')     
        except Exception as e:
            if debug_mode==True:
                plate = MockMagneticStirrer()
                plate_connected = True
            Connect_to_device_button.button_style ='danger'
            logging.error(str(datetime.datetime.now()) + '   ' + 'plate connection error, while trying all ports'+str(e))
connect_to_device()

In [29]:
#assigning the functions to the buttons
reset_graph_button.on_click(reset_graph)
save_button.on_click(save_button_clicked)
settings_button.on_click(settings_button_clicked)
Start_control_button.on_click(Start_control_button_clicked)
generate_profile_button.on_click(on_generate_profile_button_clicked)
Connect_to_device_button.on_click(on_Connect_to_device_button_clicked)
reload_prev_button.observe(on_reload_prev_button_clicked, names='value')
stirrer_button.observe(on_stirrer_button_clicked, names='value')
heater_button.observe(on_heater_button_clicked, names='value')
stirrer_speed_slider.observe(on_stirrer_slider_changed, names='value')
heater_temp_slider.observe(on_heater_slider_changed, names='value')
zoom_in_slider.observe(on_zoom_in_changed, names='value')
enable_heater_checkbox.observe(on_heater_enabler_clicked, names='value')

In [30]:
#input widgets
Temp_box = []
RampRate_box=[]
Time_to_complete_box=[]
Time_to_hold_box=[]
for i in np.arange(0,number_of_inputs):
    Temp_box.append(widgets.BoundedIntText(
        value=25,
        disabled = True,
        min=15,
        max=310
    ))
    RampRate_box.append(widgets.BoundedFloatText(
        value=1,
        disabled=True,
        min=0.01,
        max=10000.0
    ))
    Time_to_complete_box.append(widgets.BoundedFloatText(
        value=0,
        min=0,
        disabled=True,
        max=10000
    ))
    Time_to_hold_box.append(widgets.BoundedIntText(
        value=0,
        min=0,
        disabled=True,
        max=12000
    ))
Temp_box[1].disabled = False 
Time_to_hold_box[0].disabled = False
def Temp_box_value_entered(b):
    Cell_No=int(((int(b.owner.layout.grid_area.replace('widget',''))-6)/4+1))
    Time_to_complete_box[Cell_No].disabled = False
    RampRate_box[Cell_No].disabled = False
    Time_to_hold_box[Cell_No].disabled = False
    try:
        Temp_box[Cell_No+1].disabled = False
    except:
        pass
    if Temp_box[Cell_No].value < Temp_box[Cell_No-1].value:
        Time_to_complete_box[Cell_No].value = (Temp_box[Cell_No-1].value-Temp_box[Cell_No].value)/(RampRate_box[Cell_No].value/60)
    else:
        Time_to_complete_box[Cell_No].value = (Temp_box[Cell_No].value-Temp_box[Cell_No-1].value)/(RampRate_box[Cell_No].value/60)
for i in np.arange(1,number_of_inputs):    
    Temp_box[i].observe(Temp_box_value_entered, names='value')
    
    
def RampRate_box_value_entered(b):
    Cell_No=int(((int(b.owner.layout.grid_area.replace('widget',''))-6)/4))
    if Temp_box[Cell_No].value < Temp_box[Cell_No-1].value:
        Time_to_complete_box[Cell_No].value = (Temp_box[Cell_No-1].value-Temp_box[Cell_No].value)/(RampRate_box[Cell_No].value/60)
    else:
        Time_to_complete_box[Cell_No].value = (Temp_box[Cell_No].value-Temp_box[Cell_No-1].value)/(RampRate_box[Cell_No].value/60)
for i in np.arange(1,number_of_inputs):    
    RampRate_box[i].observe(RampRate_box_value_entered, names='value')
    
    
def Time_to_complete_box_value_entered(b):
    Cell_No=int(((int(b.owner.layout.grid_area.replace('widget',''))-6)/4))
    if Temp_box[Cell_No].value < Temp_box[Cell_No-1].value:
        RampRate_box[Cell_No].value = (Temp_box[Cell_No-1].value-Temp_box[Cell_No].value)/(Time_to_complete_box[Cell_No].value/60)
    else:
        RampRate_box[Cell_No].value = (Temp_box[Cell_No].value-Temp_box[Cell_No-1].value)/(Time_to_complete_box[Cell_No].value/60)
for i in np.arange(1,number_of_inputs):    
    Time_to_complete_box[i].observe(Time_to_complete_box_value_entered, names='value')
#port selector
try:
    port_selector=widgets.Dropdown(
        options=DeviceID,
        value=selected_port,
        description='Select port:',
        disabled=False,
    )
    def on_port_changed(b):
        global selected_port
        selected_port = port_selector.value
    port_selector.observe(on_port_changed, names='value')
except:
    Connect_to_device_button.disabled = True
    port_selector=widgets.Dropdown(
        options=['No ports detected'],
        value='No ports detected',
        description='Select port:',
        disabled=False,
    )

In [31]:
#Setup widgets for settings
possible_no_of_inputs = [3,4,5,6,7,8,9,10,11,12,13,14,15]
possible_length_of_intervals = [1,2,3,5,6,10,15,30,60,120,240,480]
posibble_save_intervals = [1,10,15,30,60,120,240,480]
possible_savefiles = []
save_renamed = ''
possible_savefiles = os.listdir('autosaves/')
possible_savefiles = [
    cleaned for x in possible_savefiles
    if (cleaned := ''.join([char for char in x if not char.isalpha() and char not in '._' ]))
]
possible_savefiles.append('Last save')
possible_savefiles = [
    (datetime.datetime.strptime(date, "%Y%m%d%H%M").strftime("%Y/%m/%d %H:%M") if len(date) == 12 else date)
    if len(date) == 12 and date.isdigit()
    else date
    for date in possible_savefiles
]      
# Step 1: Remove alphabetic characters and only keep digits, periods, and underscores
save_cleaned = ''.join([char for char in selected_save if not char.isalpha() and char not in '._'])
save_renamed = cleaned
# Step 2: Check if cleaned string is a 12-digit number and format it if so
if len(cleaned) == 12 and cleaned.isdigit():
    save_renamed = datetime.datetime.strptime(cleaned, "%Y%m%d%H%M").strftime("%Y/%m/%d %H:%M")

# Return the cleaned string if it's not a 12-digit number
# Widgets for General and Advanced settings
input_field_combobox = widgets.Dropdown(
    options=possible_no_of_inputs,
    value=number_of_inputs,
    description='No of Inputs:',
)

interval_combobox = widgets.Dropdown(
    options=possible_length_of_intervals,
    value=interval,
    description='Interval (s):',
)

autosave_combobox = widgets.Dropdown(
    options=possible_length_of_intervals,
    value=autosave_interval,
    description='Autosave (s):',
)

default_port_combobox = widgets.Text(
    value=default_port,
    description='Default Port:',
)

try_all_ports_combobox = widgets.Dropdown(
    options=['YES', 'NO'],
    value='YES' if try_all_ports else 'NO',
    description='Try All Ports:',
)


# Save file selection dropdown (Advanced option)
savefile_combobox = widgets.Dropdown(
    options=possible_savefiles,
    value=save_renamed,
    description='Reload Save:',
)

# Logic to handle selection changes
def on_input_change(change):
    global number_of_inputs, interval, autosave_interval,selected_save,default_port, try_all_ports
    try:
        number_of_inputs = int(change['new'])
        create_config(int(number_of_inputs),reload_prev_button.value,interval,autosave_interval,selected_save,  default_port, try_all_ports)
        logging.info(str(datetime.datetime.now()) + '   ' + 'Changed number of inputs to:' + str(number_of_inputs))
    except:
        pass
#Change logging interval
def on_interval_change(change):
    global number_of_inputs, interval,posibble_save_intervals, autosave_interval,selected_save,default_port, try_all_ports
    try:
        interval = int(change['new'])
        create_config(number_of_inputs,reload_prev_button.value,int(interval),autosave_interval,selected_save, default_port, try_all_ports)
        #This line is supposed to make sure that the autsave interval is divisible by the logging interval, but it doesnt work, IDK
        posibble_save_intervals = [1*interval,10*interval,15*interval,30*interval,60*interval,120*interval,240*interval,480*interval]
        logging.info(str(datetime.datetime.now()) + '   ' + 'Changed logging interval to: [s]' + str(interval))
    except:
        pass
#Change autosave interval
def on_autosave_change(change):
    global number_of_inputs, interval,autosave_interval, selected_save,default_port, try_all_ports
    try:
        autosave_interval = int(change['new'])
        create_config(number_of_inputs,reload_prev_button.value,int(interval),int(autosave_interval),selected_save, default_port, try_all_ports)
        logging.info(str(datetime.datetime.now()) + '   ' + 'Changed autosave interval to: [s]' + str(interval))
    except:
        pass
#select save file
def on_savefile_change(change):
    global number_of_inputs, interval,autosave_interval, selected_save,default_port, try_all_ports
    try:
        selected_save = str(change['new'])
        selected_save =(
        datetime.datetime.strptime(selected_save, "%Y/%m/%d %H:%M").strftime("%Y%m%d%H%M")
        if len(selected_save) == 16 and selected_save[4] == '/' and selected_save[7] == '/' and selected_save[13] == ':'
        else selected_save
        )
        create_config(number_of_inputs,reload_prev_button.value,int(interval),int(autosave_interval),selected_save, default_port, try_all_ports)
        logging.info(str(datetime.datetime.now()) + '   ' + 'Changed previous save to' + str(selected_save))
    except:
        pass
#Change default port
def on_default_port_change(change):
    global number_of_inputs, interval,autosave_interval, selected_save,default_port, try_all_ports
    try:
        default_port = str(change['new'])
        create_config(number_of_inputs,reload_prev_button.value,int(interval),int(autosave_interval),selected_save, default_port, try_all_ports)
        logging.info(str(datetime.datetime.now()) + '   ' + 'Changed DEFAULT PORT save to' + str(default_port))
    except:
        pass
#Change wether to try all available serial ports
def on_try_all_ports_change(change):
    global number_of_inputs, interval,autosave_interval, selected_save,default_port, try_all_ports
    try:
        if str(change['new']) == 'Yes':
            try_all_ports = True
        else:
            try_all_ports = False
        create_config(number_of_inputs,reload_prev_button.value,int(interval),int(autosave_interval),selected_save, default_port, try_all_ports)
        logging.info(str(datetime.datetime.now()) + '   ' + 'Changed try all ports to: ' + str(try_all_ports)) 
    except:
        pass    
# Assign handlers to combobox widgets
input_field_combobox.observe(on_input_change, names='value')
interval_combobox.observe(on_interval_change, names='value')
autosave_combobox.observe(on_autosave_change, names='value')
default_port_combobox.observe(on_default_port_change, names='value')
try_all_ports_combobox.observe(on_try_all_ports_change, names='value')
savefile_combobox.observe(on_savefile_change, names='value')

In [32]:
# HTML header
html_header = f"""
<div style="position: relative; height: 180px; width: 100%; background-color: transparent; overflow: hidden;">

  <div style="position: absolute; top: 35px; left: 0; right: 0; text-align: center; z-index: 10;">
    <div style="font-size: 28px; line-height: 45px;">Heat Controller for IKA Digital</div>
    <div style="font-size: 22px; line-height: 40px; margin-top: 10px;">Refreshing or closing the page will <b>terminate</b> the program; to recover, automatically reload previous save must be turned on!</div>
    <div style="font-size: 14px; line-height: 40px; margin-top: 10px;">If this page does not fully load, refresh it!</div>
  </div>
  <div style="position: absolute; top: 0px; left: 0px; right: 0px; z-index: -10; background-color: transparent;">
    <img src="gtb.png" id="rotating-logo" style="width: 100vw; height: auto;">
  </div>
</div>
"""
# Display HTML header
display(HTML(html_header))

In [33]:
### Initialize display and start code
text_fields = []
grid = widgets.GridspecLayout(number_of_inputs+5, 4, height='500px')
grid[0, 0] = widgets.HTML(value='<div style="font-size:16px; line-height:45px;"><b>Temperature target [°C]</b></div>')
grid[0, 1] = widgets.HTML(value='<div style="font-size:16px; line-height:45px;"><b>Ramp-up rate [°C/hour]</b></div>')
grid[0, 2] = widgets.HTML(value='<div style="font-size:16px; line-height:45px;"><b>Estimated time to reach target [min]</b></div>')
grid[0, 3] = widgets.HTML(value='<div style="font-size:16px; line-height:45px;"><b>Hold temperature time [min]</b></b></div>')
try:
    Temp_box[0].value = plate.probe_temperature
except:
    Temp_box[0].value = 25    
for i in np.arange(0,number_of_inputs):
    grid[i+1, 0] = Temp_box[i]
    grid[i+1, 1] = RampRate_box[i]
    grid[i+1, 2] = Time_to_complete_box[i]
    grid[i+1, 3] =Time_to_hold_box[i]
    RampRate_box[i].layout.width = '150px'  # Set width to 150 pixels
    Temp_box[i].layout.width = '150px'  # Set width to 150 pixels
    Time_to_complete_box[i].layout.width = '150px'  # Set width to 150 pixels
    Time_to_hold_box[i].layout.width = '150px'  # Set width to 150 pixels
grid[number_of_inputs+1,0] = generate_profile_button
grid[number_of_inputs+1,1] = settings_button
grid[number_of_inputs+1,2] =port_selector
grid[number_of_inputs+1,3] =Start_control_button
grid[number_of_inputs+2,0] = stirrer_speed_slider
grid[number_of_inputs+2, 1] = stirrer_button
grid[number_of_inputs+2,2] = heater_temp_slider
grid[number_of_inputs+2,3] = heater_button
grid[number_of_inputs+3,1] = info_widget
grid[number_of_inputs+3,2] = save_button
grid[number_of_inputs+3,3] = enable_heater_checkbox
grid[number_of_inputs+3,0] = Connect_to_device_button
grid[number_of_inputs+4,0:1] = zoom_in_slider
reload_prev_button.layout.width = '250px'
grid[number_of_inputs+4,2] = reload_prev_button

#grid
# Add the plot_output widget to the grid layout
#grid[number_of_inputs+5, 0:] = plot_output
#plot_thread = threading.Thread(target=update_data,daemon='True')
#plot_thread.start()
# Display the grid layout

#widgets.VBox([grid])
content_area = widgets.HBox(
    [grid],
    layout=widgets.Layout(justify_content='center', margin='0px 0px 0px 100px')  # 100px left margin
)

# Display the centered content
display(content_area)

In [34]:
#Update plot display, moved to separate grid because of display issues
grid2 = widgets.GridspecLayout(1, 4, height='600px')
plot_output = widgets.Output(layout=widgets.Layout())
grid2[0, 0:] = plot_output
plot_thread = threading.Thread(target=update_data,daemon='True')
plot_thread.start()
content_area2 = widgets.HBox(
    [grid2],
    layout=widgets.Layout(justify_content='center', margin='0px 0px 0px 100px')  # 50px left margin
)
# Display the centered content
display(content_area2)

In [35]:
#reload_data_from_previous_measurment, probly should be put in update thread
#This is a failure point on start,  possible infinit loop or graph problem, or save Current_state file overwritten before loading??
if loading_prev:
    load_previous()

In [36]:
#reset graph button
content_area3 = widgets.HBox(
    [reset_graph_button],
    layout=widgets.Layout(justify_content='center', margin='0px 0px 0px 100px')  # 50px left margin
)
display(content_area3)

In [37]:
# HTML footer
html_foot = f"""
<div style="position: relative; height: 170px; width: 90%; background-color: transparent;">
  <div style="position: absolute; bottom: 0px; right: -40px; z-index: -10;background-color: transparent;">
    <img src="green whole.png" id="footer" width="1220" height="100"
  </div>
</div>
"""

# Display the SVG with animation
HTML(html_foot)